In [15]:
import numpy as np
import random
import os

In [35]:
def apply_lif_neurons(input_data, weights, threshold= 1.0, beta=1, reset_value=0.0):
    # Assuming input_data shape is (time, channels, height, width)
    time_steps = input_data.shape[0]

    # Determine output shape after convolution
    output_shape = (time_steps, weights.shape[0],  # (time, output_channels, height, width)
                    input_data.shape[2] - weights.shape[2] + 1,
                    input_data.shape[3] - weights.shape[3] + 1)

    # Initialize output spikes
    output_spikes = np.zeros(output_shape)  # Shape: (time, output_channels, height, width)

    # Initialize membrane potential for this timestep
    membrane_potential = np.zeros((time_steps, weights.shape[0], output_shape[2], output_shape[3]))  # Shape: (output_channels, height, width)

    # Perform convolution for the current timestep
    for out_channel in range(output_shape[1]):
        for i in range(output_shape[2]):
            for j in range(output_shape[3]):
              for t in range(time_steps):
                # Calculate convolution output at this stride
                conv_output = np.sum(input_data[t, :, i:i+weights.shape[2], j:j+weights.shape[3]] * weights[out_channel])

                # Update membrane potential with leaky integration
                if t == 0:
                  membrane_potential[t, out_channel, i, j] = beta * membrane_potential[t, out_channel, i, j] + conv_output
                else:
                  membrane_potential[t, out_channel, i, j] = beta * membrane_potential[t-1, out_channel, i, j] + conv_output

                # Generate spike if membrane potential exceeds threshold
                if membrane_potential[t, out_channel, i, j] >= threshold:
                    output_spikes[t, out_channel, i, j] = 1
                    # Reset membrane potential where spike occurred
                    # membrane_potential[t,out_channel, i, j] = membrane_potential[t,out_channel, i, j]-threshold
                    membrane_potential[t,out_channel, i, j] = reset_value

    return output_spikes, membrane_potential


In [56]:
def conv_results(row, col, channel, ts):
    result = []
    int_result = []  # To store cumulative sums
    
    for step in range(ts):
        conv_output = input_data[step, :, row:row+quant_weight_int.shape[2], col:col+quant_weight_int.shape[3]] * quant_weight_int[channel]
        print(np.sum(conv_output))
        result.append(np.sum(conv_output))
#         print(conv_output)

         # Calculate the cumulative sum
        if step != 0:
            int_result.append(result[step] + int_result[step-1])
        else:
            int_result.append(result[step])
    
    return int_result

In [37]:
base_path = 'C:/Users/mayaa/OneDrive/Documents/Master_project/vhdl_may'
quant_weight_int = np.load(base_path + '/current_test/npy/quant_weight_int.npy')

timesteps = 3
sample_input = np.load(base_path + '/current_test/npy/in_spk_8b0.npy')
channels, rows, cols = sample_input.shape
input_data = np.empty((timesteps, channels, rows, cols))

for step in range(timesteps):
    input_data[step] = np.load(base_path + f'/current_test/npy/in_spk_8b{step}.npy')
    
output_spikes, membrane_potential = apply_lif_neurons(input_data, quant_weight_int, threshold= 101)

In [64]:
result = []
int_result = []

int_result = conv_results(row=2, col=2, channel=2, ts=3)
print(f'Integrated over timesteps {int_result}')

13.0
40.0
50.0
Integrated over timesteps [13.0, 53.0, 103.0]
